In [2]:
from transformers import BertModel
import torch
import torch.nn.functional as F
import sentencepiece as spm

In [4]:
TOKENIZER_MODEL = "../tokenizer/unigram_32000_0.9995.model"
MODEL_DIR = "../SinBERT_v02"


sp = spm.SentencePieceProcessor()
sp.load(TOKENIZER_MODEL)


model = BertModel.from_pretrained(MODEL_DIR, add_pooling_layer=False)
model.eval()


print("✓ Tokenizer and model loaded")

✓ Tokenizer and model loaded


In [6]:
def embed_sentence(text: str):
    # Convert a Sinhala sentence into a fixed-size semantic embedding.
    ids = torch.tensor([sp.encode(text, out_type=int)])
    mask = (ids != sp.pad_id()).unsqueeze(-1)


    with torch.no_grad():
        out = model(ids)


    token_embeddings = out.last_hidden_state
    mean_embedding = (token_embeddings * mask).sum(dim=1) / mask.sum(dim=1)


    return mean_embedding.squeeze(0) # shape: [hidden_size]

In [ ]:
s1 = "කෘත්‍රිම බුද්ධිය ශ්‍රී ලංකාවේ අනාගතය වෙනස් කරයි"
s2 = "කෘත්‍රිම බුද්ධිය ශ්‍රී ලංකාවේ අනාගතයට විශාල බලපෑමක් ඇති කරයි"

In [8]:
e1 = embed_sentence(s1)
e2 = embed_sentence(s2)


print("Embedding shape:", e1.shape)

Embedding shape: torch.Size([384])


In [9]:
similarity = F.cosine_similarity(e1, e2, dim=0)
print(f"Semantic similarity score: {similarity.item():.4f}")

Semantic similarity score: 0.6953


In [14]:
pairs = [
(
"කෘත්‍රිම බුද්ධිය අනාගතය වෙනස් කරයි",
"කෘත්‍රිම බුද්ධිය අනාගතයට බලපායි",
),
(
"කෘත්‍රිම බුද්ධිය අනාගතය වෙනස් කරයි",
"අද කාලගුණය ඉතා උණුසුම්ය",
),
(
"ශ්‍රී ලංකාවේ ආර්ථිකය වර්ධනය වී ඇත",
"ශ්‍රී ලංකාවේ ආර්ථිකය දුර්වල වී ඇත",
),
]


for a, b in pairs:
    sim = F.cosine_similarity(embed_sentence(a), embed_sentence(b), dim=0)
    print(f"{sim.item():.3f} | {a} <> {b}")

0.780 | කෘත්‍රිම බුද්ධිය අනාගතය වෙනස් කරයි <> කෘත්‍රිම බුද්ධිය අනාගතයට බලපායි
0.472 | කෘත්‍රිම බුද්ධිය අනාගතය වෙනස් කරයි <> අද කාලගුණය ඉතා උණුසුම්ය
0.959 | ශ්‍රී ලංකාවේ ආර්ථිකය වර්ධනය වී ඇත <> ශ්‍රී ලංකාවේ ආර්ථිකය දුර්වල වී ඇත


In [ ]:
# --------------------------------------------------
# Interpretation guide:
# 0.80 – 1.00 : Very similar meaning
# 0.60 – 0.80 : Related / paraphrase
# 0.40 – 0.60 : Weakly related
# < 0.40 : Different meaning